In [ ]:
#Import library
import numpy as np
from keras.layers import Conv2D,Input,ZeroPadding2D
from keras.layers import BatchNormalization ,Activation, MaxPooling2D
from keras.layers import Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model,load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils  import shuffle
from keras.optimizers  import RMSprop
from keras.callbacks import TensorBoard, ModelCheckpoint


In [ ]:
#Set the directory for dataset
train_path = "/content/drive/My Drive/Colab Notebooks/data/Mask dataset/train"
test_path = "/content/drive/My Drive/Colab Notebooks/data/Mask dataset/test"

Lets make CNN

If inputs are of more than two classes..then we use softmax activation function but here are two inputs so we use sigmoid function.

We have binary class input so we use binary entrophy function in loss ... otherwise more than two we use Categorical cross entropy



In [ ]:
#Intialize the CNN
classifier=Sequential()

#Step 1-  Convolution
classifier.add(Conv2D(32,(3,3),input_shape=(128,128,3),activation="relu"))

#step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

#step 3 - Flatten
classifier.add(Flatten())

#step 4- Fully Connected Layer
classifier.add(Dense(units=128 ,activation="relu"))

#Output layer
classifier.add(Dense(units=1 ,activation="sigmoid"))

#Lets calculate loss function
classifier.compile(optimizer="adam",loss="binary_crossentropy",metrics=["acc"])

 

Preprocess images

In [ ]:
#Train 
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
                                                    train_path, 
                                                    batch_size=90, 
                                                    class_mode='binary',
                                                    target_size=(128, 128))
                                                    
#Validation
validation_datagen = ImageDataGenerator(rescale=1.0/255)
                                                    
validation_generator = validation_datagen.flow_from_directory(
                                                         test_path, 
                                                         batch_size=90, 
                                                         class_mode='binary',
                                                         target_size=(128, 128))



In [ ]:
#Prevent overfitting
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=True,
                             mode='auto')

In [ ]:
#Lets fit model 
model=classifier.fit_generator(
    train_generator,
    epochs=10,
    validation_data=validation_generator
  

)




In [ ]:
import os
import datetime
# Create a function to save a model
def save_model(model, suffix=None):
  """
  Saves a given model in a models directory and appends a suffix (str)
  for clarity and reuse.
  """
  # Create model directory with current time
  modeldir = os.path.join("/content/drive/My Drive/Colab Notebooks/",
                          datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
  model_path = modeldir + "-" + suffix + ".h5" # save format of model
  print(f"Saving model to: {model_path}...")
  model.save(model_path)
  return model_path

In [ ]:
save_model(classifier,"model")